In [49]:
# import modules
import numpy as np

# import plotting modules
import matplotlib
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib inline

from IPython.display import Latex

%matplotlib inline

from matplotlib import gridspec
from NFW_Z import NFW_Z

In [50]:
class BestNFWFits:
    
    
    def __init__(self, Mv):
        
        """Initiate the class with a known virial mass of the halo.
        Input: virial mass in solar masses."""
        
        self.Rvir = Mv


    def find_dens_profile(self, raw_radius_data, radius_array, bin_size):
        
        """Derive an AVERAGE dark matter density profile from raw data.
        Inputs:
        1) raw_radius_data: array of the radii at which dark matter particles are found, based on provided data.
        2) radius_array: array of the radii of imaginary spherical shells centered on the galactic center.
        3) bin_size: integer that best matches the image resolution.
        Output: density_profile (the AVERAGE dark matter density profile of the data)"""
        
        density_profile = np.zeros(np.size(radius_array))
        i = 0
        for radius_value in radius_array: # enclosed density
            particles = np.where((raw_radius_data > radius_value) & (raw_radius_data < (radius_value + bin_size)))
            how_many_particles = np.size(particles)
            shell_volume = 4/3*np.pi*((radius_value + bin_size)**3 - radius_value**3)
            density_profile[i] = how_many_particles / shell_volume * 500 #500 is 500 M_Sun, the mass per particle
            i += 1
        return density_profile
    
    
    def final_plot(self, quoted_Mvir, best_cvir, increased_Mvir_1, best_cvir_1, increased_Mvir_2, best_cvir_2, \
                   redshift, raw_radius_data, radius_array, bin_size):
        
        """Make a ratio plot that compares the actual density profile generated from data with the best NFW fits
        that vary in M_vir and c_vir.
        Inputs:
        1) quoted_Mvir: the virial mass quoted in the provided data.
        2) best_cvir: the c_vir value that gives the best NFW fit to the density profile based on the provided virial mass.
        3) increased_Mvir_1: the virial mass quoted in the provided data, increased by a certain increment.
        4) best_cvir_1: the c_vir value that gives the best NFW fit to the density profile based on the previous increased mass.
        5) increased_Mvir_2: the virial mass quoted in the provided data, increased by twice the abovementioned increment.
        6) best_cvir_2: the c_vir value that gives the best NFW fit to the density profile based on the previous increased mass.
        7) redshift: the redshift at which the data was measured.
        8) raw_radius_data: array of the radii at which dark matter particles are found, based on provided data.
        9) radius_array: array of the radii of imaginary spherical shells centered on the galactic center.
        10) bin_size: integer that best matches the image resolution.
        Output: the ratio plot, of course!"""
        
        # take in the values
        NFW_1 = NFW_Z(quoted_Mvir) 
        NFW_2 = NFW_Z(increased_Mvir_1)
        NFW_3 = NFW_Z(increased_Mvir_2) 
        
        # extract the density profile array from find_dens_profile
        density_profile = self.find_dens_profile(raw_radius_data, radius_array, bin_size)

        # plot everything
        fig = plt.figure(figsize=(8, 10)) 
        gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1]) # row, columns, widths

        ax0 = plt.subplot(gs[0]) # top plot
        ax1 = plt.subplot(gs[1]) # bottom plot 

        fig.subplots_adjust(hspace=0.1)  

        ax0.loglog(radius_array, density_profile, 'black', lw=2, label="data")
        ax0.loglog(radius_array, NFW_1.rho(redshift, radius_array, c = best_cvir), '#EC4E20', lw=2, linestyle='-.', \
                  label="NFW1; $M_{vir}$ = quoted_Mvir, $c_{vir}$ = best_cvir")
        ax0.loglog(radius_array, NFW_2.rho(redshift, radius_array, c = best_cvir_1), '#FF9505', lw=2, linestyle='-.', \
                  label="NFW2; $M_{vir}$ = increased_Mvir_1, $c_{vir}$ = best_cvir_1")
        ax0.loglog(radius_array, NFW_2.rho(redshift, radius_array, c = best_cvir_2), '#016FB9', lw=2, linestyle='-.', \
                  label="NFW3; $M_{vir}$ = increased_Mvir_2, $c_{vir}$ = best_cvir_2")

        ax0.legend(bbox_to_anchor=(1.02, 1.01), loc='upper left')

        ax1.plot(radius_array, density_profile / NFW_1.rho(redshift, radius_array, c = best_cvir), \
                 '#EC4E20', linestyle='-.', lw=2, label="data/NFW1")
        ax1.plot(radius_array, density_profile / NFW_2.rho(redshift, radius_array, c = best_cvir_1), \
                 '#FF9505', linestyle='-.', lw=2, label="data/NFW2")
        ax1.plot(radius_array, density_profile / NFW_3.rho(redshift, radius_array, c = best_cvir_2), \
                 '#016FB9', linestyle='-.', lw=2, label="data/NFW3")
        
        ax1.legend(bbox_to_anchor=(1.02, 0), loc='lower left')

        ax0.minorticks_on()

        ax0.set_title('Plot Title', fontsize=20)
        ax0.set_xticklabels([]) # ignore the x label
        ax0.set_ylabel(r' $\rho$ [M$_\odot$/kpc$^3$] ', fontsize=20)
        ax0.tick_params(axis='both', which='both', direction='in',length=6, width=2)

        plt.xscale('log')

        ax1.set_ylabel('Ratio',fontsize=20)
        ax1.set_xlabel('r (kpc)', fontsize=20)

        ax1.grid()
        ax1.minorticks_on()
        ax1.tick_params(axis='both', which='both', direction='in',length=6, width=1)

        plt.rcParams.update({'font.size': 15})
        
        # lo and behold!
        plt.show()